In [5]:
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import pandas as pd
import serial

import time
import os
import shutil
import sys
from datetime import datetime
print(sys.version)

3.5.4 |Anaconda custom (64-bit)| (default, Sep 19 2017, 08:15:17) [MSC v.1900 64 bit (AMD64)]


In [6]:
# list ports

def serial_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result


# I'm using COM3 and COM5 arduinos
# COM4 is for the arduino controlling the cameras
serial_ports()

['COM3', 'COM5', 'COM6']

In [39]:
# ser1.close()

In [40]:
# connect 
PORT1 = "COM6"
connected1 = False
ser1 = serial.Serial(PORT1,9600)
while not connected1:
    serin1 = ser1.read()
    connected1 = True
    print("connected to arduino on " + PORT1)

connected to arduino on COM6


In [41]:
ser1.readline()

b'etup complete\r\n'

In [196]:
written = ser1.write("f".encode("utf-8"))


In [412]:
written = ser1.write("b".encode("utf-8"))

In [38]:
"b".encode("utf-8")
int.from_bytes(b'b', byteorder='big') # this is what the arduino will see

98

In [35]:
int.from_bytes(b'f', byteorder='big')

102